In [1]:


#pdf plumber
#csv.preview
import pandas as pd
#import unicode character database
import unicodedata
#import regular expression operations
import re

#import natural language toolkit
import nltk
from nltk.corpus import words
#import our aquire
from acquire import *


#import our stopwords list
from nltk.corpus import stopwords
from copy import deepcopy








def split_apply_join(funct,listobj):
    'helperfuction letters'

    mapped=map(funct, listobj)
    mapped=list(mapped)
    mapped=''.join(mapped)
  
    return mapped






def stopfilter(text,stop_words_extend_reduce=["'"]):
    'we use symmetric difference so if a is already in stop words then it will be added to our third set else our third set will be missing it'
    #create oujr english stopwords list

    stops = set(stopwords.words('english'))
    stop_words_extend_reduce=set(stop_words_extend_reduce)
    stops=stops.symmetric_difference(stop_words_extend_reduce)

    # stops=(stops|stop_words_extend)-exclude_words
    #another way
    
    filtered=list(filter((lambda x: x not in stops), text.split()))
    filtered=' '.join(filtered)

    return filtered









def basic_clean(text,regexfilter=r'[^a-z0-9\'\s]'):
    '''   
    Filters out all special characters if you need to edit then supply a new regex filter 
    
    
    
    
    '''
    #make a copy and begin to transform it
    newtext = text.lower()

    #encode into ascii then decode
    newtext = unicodedata.normalize('NFKD', newtext)\
    .encode('ascii', 'ignore')\
    .decode('utf-8')

    #use re.sub to remove special characters
    newtext = re.sub(fr'{regexfilter}', ' ', newtext)

    
    


    return newtext

 


def tokenizer(text,regexfilter=r'[^a-z0-9\'\s]'):
    ''' 
    For a large file just save it locally
    
    
    
    
    
    '''
    newtext=basic_clean(text,regexfilter=regexfilter)
    #make ready tokenizer object
    tokenize = nltk.tokenize.ToktokTokenizer()
    #use the tokenizer
    newtext = tokenize.tokenize(newtext, return_str=True)
    return newtext





    
    
    
    


def stemmed(text,regexfilter=r'[^a-z0-9\'\s]'):
    '''    
    Takes text, tokenizes it, stems it
    
    
    
    
    
    '''
    total=list(pd.read_pickle('words.pkl'))


    
    #make ready porter stemmer object
    newtext=tokenizer(text,regexfilter=regexfilter)
    ps = nltk.porter.PorterStemmer()
    stemmedlist=split_apply_join(ps.stem,newtext)
    # since the average word lenght in English is 4.7 characters we will apply a conservative estimate and drop any word that is larger than 8 characters as it is likely not a word
    # we also recursivley took the set of all words generated then compared that to nltk.corpus.words.words() and used that list as filter this is where total comes from

    stemfiltered=list(filter(lambda x:(len(x)>1 and len(x)<9 and x.isalpha()==True and (x in  total)), stemmedlist.split()))
    stemfiltered=' '.join(stemfiltered)
 
    stemfiltered=basic_clean(stemfiltered,regexfilter=regexfilter)

    return stemfiltered




def lemmatizor(text,regexfilter=r'[^a-z0-9\'\s]'):
    '''    
    
      Takes text, tokenizes it, lemmatizes it
    
    
    
    
    
    '''
    total=list(pd.read_pickle('words.pkl'))
    

    #make ready the lemmatizer object
    newtext=tokenizer(text,regexfilter=regexfilter)
    wnl = nltk.stem.WordNetLemmatizer()
    lemmatized=split_apply_join(wnl.lemmatize,newtext)

    # since the average word lenght in English is 4.7 characters we will apply a conservative estimate and drop any word that is larger than 8 characters as it is likely not a word
    # we also recursivley took the set of all words generated then compared that to nltk.corpus.words.words() and used that list as filter this is where total comes from

    lemmafiltered=list(filter(lambda x: (len(x)>1 and len(x)<9 and x.isalpha()==True and (x in  total)), lemmatized.split()))
    lemmafiltered=' '.join(lemmafiltered)
  
    lemmafiltered=basic_clean(lemmafiltered,regexfilter=regexfilter)

    return lemmafiltered




def dictlist_super_NLP_comp(dictlist,regexfilter=r'[^a-z0-9\'\s]',stop_words_extend_reduce=["'"]):
    ''
    ndictlist=deepcopy(dictlist)
    mapper=[]
    interestingkeys=list(ndictlist.keys())
    for i in range(0,len(ndictlist)):           
            k=interestingkeys[i]
            text=ndictlist.get(k)         
            org={f'org':text}
            clean=basic_clean(text,regexfilter=regexfilter)
            cleaned=({f'cleaned':clean})
            lemmatized=lemmatizor(text,regexfilter)
            stopfilteredlemitezed=stopfilter(lemmatized,stop_words_extend_reduce=stop_words_extend_reduce)
            lemma={f'lemmatized':stopfilteredlemitezed}
            stem=stemmed(text,regexfilter)
            stopfilteredstem=stopfilter(stem,stop_words_extend_reduce=stop_words_extend_reduce)
            stemma={f'stemmed':stopfilteredstem}
            mapper.append({k:dict(**org,**cleaned,**lemma,**stemma)})              
               


  
       
    return mapper


In [2]:
datadf=pd.read_json('data.json')

len(datadf)

# df_C=datadf[datadf['language']=='C'];df_C.head(n=30)


# df_Python=datadf[datadf['language']=='Python'];df_Python.head(n=30)


# df_Java=datadf[datadf['language']=='Java'];df_Java.head(n=30)





all_articles=(datadf['readme_contents']).to_dict()



In [3]:
newdf=dictlist_super_NLP_comp(all_articles)


In [4]:
total=list(pd.read_pickle('words.pkl'))
total

['aa',
 'ability',
 'able',
 'abort',
 'abstract',
 'accent',
 'accept',
 'accepted',
 'access',
 'account',
 'accuracy',
 'accurate',
 'achieve',
 'acorn',
 'across',
 'act',
 'acting',
 'action',
 'activate',
 'active',
 'actively',
 'activity',
 'actual',
 'actually',
 'ad',
 'adapt',
 'adaptor',
 'add',
 'added',
 'addition',
 'additive',
 'address',
 'adjust',
 'adoption',
 'advance',
 'advanced',
 'advice',
 'advised',
 'ae',
 'affect',
 'affine',
 'affinity',
 'age',
 'agent',
 'agnostic',
 'ago',
 'agora',
 'ahead',
 'aid',
 'aim',
 'aka',
 'al',
 'alert',
 'alias',
 'align',
 'alive',
 'allocate',
 'allow',
 'almost',
 'along',
 'alpha',
 'already',
 'also',
 'alt',
 'although',
 'altitude',
 'always',
 'amateur',
 'amazing',
 'among',
 'amount',
 'amplify',
 'anaconda',
 'analogy',
 'analysis',
 'analyze',
 'android',
 'angular',
 'animated',
 'another',
 'answer',
 'ant',
 'anti',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'apache',
 'apart',
 'apogee',
 'apparent',
 '

In [5]:

a=pd.DataFrame(newdf[0]).T
for i in range(1,len(newdf)):
    b=pd.DataFrame(newdf[i]).T
    a=pd.merge(a,b,how='outer',right_index=False,left_index=False)


    


In [6]:
df=pd.merge(datadf,a,how='left',right_index=True,left_index=True)
display(df.head(),datadf.head())## Verify merge
datadf=df #finish prepped


,repo,language,readme_contents,cleaned,lemmatized,org,stemmed
0,cstack/db_tutorial,C,# Let's Build a Simple Database\n\n[View rende...,let's build a simple database\n\n view rende...,let build simple view tutorial io tutorial run...,# Let's Build a Simple Database\n\n[View rende...,let build simple view tutorial io tutorial run...
1,rui314/chibicc,C,# chibicc: A Small C Compiler\n\n(The old mast...,chibicc a small c compiler\n\n the old mast...,small compiler old master old tree old branch ...,# chibicc: A Small C Compiler\n\n(The old mast...,small compiler old master old tree old branch ...
2,nelhage/reptyr,C,"reptyr - A tool for ""re-ptying"" programs.\n===...",reptyr a tool for re ptying programs \n ...,tool utility taking running program new termin...,"reptyr - A tool for ""re-ptying"" programs.\n===...",tool utility taking running program new termin...
3,EyalAr/lwip,C,[![Version](http://img.shields.io/npm/v/lwip.s...,version http img shields io npm v lwip s...,version io package build status travis branch ...,[![Version](http://img.shields.io/npm/v/lwip.s...,version io package build status travis branch ...
4,ibireme/yyjson,C,\n# Introduction\n\n[![Build](https://flat.bad...,\n introduction\n\n build https flat bad...,build flat net status master flat net io licen...,\n# Introduction\n\n[![Build](https://flat.bad...,build flat net status master flat net io licen...


,repo,language,readme_contents
0,cstack/db_tutorial,C,# Let's Build a Simple Database\n\n[View rende...
1,rui314/chibicc,C,# chibicc: A Small C Compiler\n\n(The old mast...
2,nelhage/reptyr,C,"reptyr - A tool for ""re-ptying"" programs.\n===..."
3,EyalAr/lwip,C,[![Version](http://img.shields.io/npm/v/lwip.s...
4,ibireme/yyjson,C,\n# Introduction\n\n[![Build](https://flat.bad...


In [7]:

reorder=['language',
'repo',
 'readme_contents',
 'cleaned',
 'stemmed',
 'lemmatized']
  
datadf=datadf[reorder]



In [8]:

datadf
pd.to_pickle(datadf,'cleanpickle.pkl')




